<a href="https://colab.research.google.com/github/aminfadaei116/Deep-Learning-Course/blob/master/CA2/Q2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import cv2
from scipy.io import loadmat
import numpy as np
import tqdm
import torch
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
img_tensor = torch.load('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW2-DataSet/Q2-DataSet/hazelnut/hazelnut_good_tensor.pt')
img_tensor = img_tensor.to(device)
crack_tensor = torch.load('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW2-DataSet/Q2-DataSet/hazelnut/hazelnut_crack_tensor.pt')
crack_tensor = crack_tensor.to(device)
cut_tensor = torch.load('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW2-DataSet/Q2-DataSet/hazelnut/hazelnut_cut_tensor.pt')
cut_tensor = cut_tensor.to(device)
hole_tensor = torch.load('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW2-DataSet/Q2-DataSet/hazelnut/hazelnut_hole_tensor.pt')
hole_tensor = hole_tensor.to(device)
print_tensor = torch.load('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW2-DataSet/Q2-DataSet/hazelnut/hazelnut_print_tensor.pt')
print_tensor = print_tensor.to(device)

In [ ]:
crack_mask = torch.load('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW2-DataSet/Q2-DataSet/hazelnut/small_hazelnut_crack_mask.pt')
crack_mask = crack_mask.to(device)
cut_mask = torch.load('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW2-DataSet/Q2-DataSet/hazelnut/small_hazelnut_cut_mask.pt')
cut_mask = cut_mask.to(device)
hole_mask = torch.load('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW2-DataSet/Q2-DataSet/hazelnut/small_hazelnut_hole_mask.pt')
hole_mask = hole_mask.to(device)
print_mask = torch.load('/content/drive/My Drive/Deep Learning Course Spring 99/DataSets/HW2-DataSet/Q2-DataSet/hazelnut/small_hazelnut_print_mask.pt')
print_mask = print_mask.to(device)

In [ ]:
# makes it easy to write user-friendly command-line interfaces
import argparse
# contains data structures for multi-dimensional tensors
# and mathematical operations over theme
import torch
# helps us in creating and training of the neural network.
import torch.nn as nn
# contains all the functions in the torch.nn library
import torch.nn.functional as F
# implements various optimization algorithms
import torch.optim as optim
# contains datasets with almost similar API
from torchvision import datasets
# contains common image transformations
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
# provides several methods to adjust the learning rate based on the number of epochs
from torch.optim.lr_scheduler import StepLR

In [ ]:
class HAZELNUT_IMG(Dataset):
  def __init__(self, img):
    self.img_data = img.view(-1, 3, 128, 128)/256
    self.length = img.shape[0]

  def __getitem__(self, index):
    return self.img_data[index]

  def __len__(self):
    return self.length


In [ ]:
class HAZELNUT_MASK(Dataset):
  def __init__(self, img):
    self.img_data = img.view(-1, 128, 128)
    self.img_data[self.img_data >= 0.5] = 1
    self.img_data[self.img_data < 0.5] = 0
    self.length = img.shape[0]

  def __getitem__(self, index):
    return self.img_data[index]

  def __len__(self):
    return self.length


In [ ]:
MyTrainDataSet = HAZELNUT_IMG(img_tensor)
MyCrackDataSet = HAZELNUT_IMG(crack_tensor)
MyCutDataSet = HAZELNUT_IMG(cut_tensor)
MyHoleDataSet = HAZELNUT_IMG(hole_tensor)
MyPrintDataSet = HAZELNUT_IMG(print_tensor)

In [ ]:
MyCrackMask = HAZELNUT_MASK(crack_mask)
MyCutMask = HAZELNUT_MASK(cut_mask)
MyHoleMask = HAZELNUT_MASK(hole_mask)
MyPrintMask = HAZELNUT_MASK(print_mask)

In [ ]:
TrainData = torch.utils.data.DataLoader(dataset=MyTrainDataSet, batch_size=32, shuffle=True)

In [ ]:
class CNN_Net(nn.Module):
  def __init__(self):
    self.loss_graphT = []    
    self.Method1AccCrack = []
    self.Method2AccCrack = []
    self.Method3AccCrack = []
    self.Method1AccCut = []
    self.Method2AccCut = []
    self.Method3AccCut = []
    self.Method1AccHole = []
    self.Method2AccHole = []
    self.Method3AccHole = []
    self.Method1AccPrint = []
    self.Method2AccPrint = []
    self.Method3AccPrint = []
    super(CNN_Net, self).__init__()
    self.encoder = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4, stride=2, padding=1),
      nn.BatchNorm2d(32, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.Conv2d(in_channels=32, out_channels=32, kernel_size=4, stride=2, padding=1),
      nn.BatchNorm2d(32, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2, padding=1),
      nn.BatchNorm2d(64, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1),
      nn.BatchNorm2d(128, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.Conv2d(in_channels=32, out_channels=100, kernel_size=8, stride=1, padding=0),
      nn.BatchNorm2d(100, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False)
    )
    self.decoder = nn.Sequential(
      nn.ConvTranspose2d(in_channels = 100, out_channels = 32, kernel_size=8, stride=1),
      nn.BatchNorm2d(32, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.ConvTranspose2d(in_channels = 32, out_channels = 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.ConvTranspose2d(in_channels = 64, out_channels = 128, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(128, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.ConvTranspose2d(in_channels = 128, out_channels = 64, kernel_size=4, stride=2, padding=1),
      nn.BatchNorm2d(64, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.ConvTranspose2d(in_channels = 64, out_channels = 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=4, stride=2, padding=1),
      nn.BatchNorm2d(32, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=4, stride=2, padding=1),
      nn.LeakyReLU(negative_slope=0.01, inplace=False),
      nn.ConvTranspose2d(in_channels=32, out_channels=3, kernel_size=4, stride=2, padding=1),
      nn.Sigmoid()
    )


  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x

  def Train_net(self, learning_rate, epoch_number):
    self.train()
    optimizer = optim.Adam(MyNetwork.parameters(), lr=learning_rate)
    loss_function = nn.MSELoss()
    EPOCHS = epoch_number
    for epoch in range(EPOCHS):
      for i, (inputs) in enumerate(TrainData):
        batch_X = inputs
        optimizer.zero_grad()
        output = MyNetwork(batch_X)
        loss = loss_function(output, batch_X)
        self.loss_graphT.append(loss.item())
        loss.backward()
        optimizer.step()
        print("epoch: ", epoch, " loss for batch: ", loss)
      self.cal_acc_print()
      self.cal_acc_crack()
      self.cal_acc_cut()
      self.cal_acc_hole()
      

  def Test_single(self, number, Current_DataSet):
    MyNetwork.eval()
    with torch.no_grad():
      result =  MyNetwork.forward(Current_DataSet[number:number+1])
    return result

  def Test_single_img(self, number, Current_DataSet):
    MyNetwork.eval()
    with torch.no_grad():
      result =  MyNetwork.forward(Current_DataSet[number:number+1])
    plt.imshow(result.view(128, 128, 3).cpu().detach().numpy())
    plt.show()
    plt.imshow(Current_DataSet[number].view(128, 128, 3).cpu().detach().numpy())
    plt.show()
    return result

  def plot_res(self):
    plt.plot(np.arange(1,len(self.loss_graphT)+1), self.loss_graphT)
    plt.ylabel('loss of Train')
    plt.xlabel("number steps")
    plt.grid(True)
    plt.show()

    plt.plot(np.arange(1,len(self.Method1AccCrack)+1), self.Method1AccCrack)
    plt.ylabel('Method1AccCrack accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()

    plt.plot(np.arange(1,len(self.Method2AccCrack)+1), self.Method2AccCrack)
    plt.ylabel('Method2AccCrack accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()

    plt.plot(np.arange(1,len(self.Method3AccCrack)+1), self.Method3AccCrack)
    plt.ylabel('Method3AccCrack accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()

    plt.plot(np.arange(1,len(self.Method1AccCut)+1), self.Method1AccCut)
    plt.ylabel('Method1AccCut accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()
    
    plt.plot(np.arange(1,len(self.Method2AccCut)+1), self.Method2AccCut)
    plt.ylabel('Method2AccCut accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()
      
    plt.plot(np.arange(1,len(self.Method3AccCut)+1), self.Method3AccCut)
    plt.ylabel('Method3AccCut accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()

    plt.plot(np.arange(1,len(self.Method1AccHole)+1), self.Method1AccHole)
    plt.ylabel('Method1AccHole accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()
      
    plt.plot(np.arange(1,len(self.Method2AccHole)+1), self.Method2AccHole)
    plt.ylabel('Method2AccHole accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()

    plt.plot(np.arange(1,len(self.Method3AccHole)+1), self.Method3AccHole)
    plt.ylabel('Method3AccHole accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()

    plt.plot(np.arange(1,len(self.Method1AccPrint)+1), self.Method1AccPrint)
    plt.ylabel('Method1AccPrint accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()

    plt.plot(np.arange(1,len(self.Method2AccPrint)+1), self.Method2AccPrint)
    plt.ylabel('Method2AccPrint accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()

    plt.plot(np.arange(1,len(self.Method3AccPrint)+1), self.Method3AccPrint)
    plt.ylabel('Method3AccPrint accuracy')
    plt.xlabel("Epochs")
    plt.grid(True)
    plt.show()


  def cal_acc_crack(self):
    Method1Acc = []
    Method2Acc = []
    Method3Acc = []
    for num in range(18):
      pred_img = self.Test_single(num, MyCrackDataSet)
      A = (pred_img - MyCrackDataSet[num]).view(128, 128, 3).cpu().detach().numpy()
      B = (MyCrackDataSet[num] - pred_img).view(128, 128, 3).cpu().detach().numpy()
      threshold = 0.1
      A[A>=threshold] = 1
      A[A<threshold] = 0
      B[B>=threshold] = 1
      B[B<threshold] = 0
      result = (A + B)
      #binary_res = np.logical_and(result[:,:,0], result[:,:,1], result[:,:,2])
      binary_res = np.max(result, axis=2)

      True_mask = MyCrackMask[num].cpu().detach().numpy()
      res = (2 * True_mask-binary_res)
      maskTpreF = np.count_nonzero(res == 2)
      maskTpreT = np.count_nonzero(res == 1)
      maskFpreF = np.count_nonzero(res == 0)
      maskFpreT = np.count_nonzero(res == -1.0)

      Method1Acc.append(maskTpreT/(maskTpreT + maskTpreF))
      Method2Acc.append(maskTpreT/(maskTpreT + maskFpreT))
      Method3Acc.append((maskTpreT + maskFpreF)/(maskTpreT + maskFpreF + maskTpreF + maskFpreT))

    self.Method1AccCrack.append(sum(Method1Acc)/len(Method1Acc))
    self.Method2AccCrack.append(sum(Method2Acc)/len(Method2Acc))
    self.Method3AccCrack.append(sum(Method3Acc)/len(Method3Acc))

  def cal_acc_cut(self):
    Method1Acc = []
    Method2Acc = []
    Method3Acc = []
    for num in range(17):
      pred_img = self.Test_single(num, MyCutDataSet)
      A = (pred_img - MyCutDataSet[num]).view(128, 128, 3).cpu().detach().numpy()
      B = (MyCutDataSet[num] - pred_img).view(128, 128, 3).cpu().detach().numpy()
      threshold = 0.1
      A[A>=threshold] = 1
      A[A<threshold] = 0
      B[B>=threshold] = 1
      B[B<threshold] = 0
      result = (A + B)
      #binary_res = np.logical_and(result[:,:,0], result[:,:,1], result[:,:,2])
      binary_res = np.max(result, axis=2)


      True_mask = MyCutMask[num].cpu().detach().numpy()
      res = (2 * True_mask-binary_res)
      maskTpreF = np.count_nonzero(res == 2)
      maskTpreT = np.count_nonzero(res == 1)
      maskFpreF = np.count_nonzero(res == 0)
      maskFpreT = np.count_nonzero(res == -1.0)

      Method1Acc.append(maskTpreT/(maskTpreT + maskTpreF))
      Method2Acc.append(maskTpreT/(maskTpreT + maskFpreT))
      Method3Acc.append((maskTpreT + maskFpreF)/(maskTpreT + maskFpreF + maskTpreF + maskFpreT))

    self.Method1AccCut.append(sum(Method1Acc)/len(Method1Acc))
    self.Method2AccCut.append(sum(Method2Acc)/len(Method2Acc))
    self.Method3AccCut.append(sum(Method3Acc)/len(Method3Acc))

  def cal_acc_hole(self):
    Method1Acc = []
    Method2Acc = []
    Method3Acc = []
    for num in range(18):
      pred_img = self.Test_single(num, MyHoleDataSet)
      A = (pred_img - MyHoleDataSet[num]).view(128, 128, 3).cpu().detach().numpy()
      B = (MyHoleDataSet[num] - pred_img).view(128, 128, 3).cpu().detach().numpy()
      threshold = 0.1
      A[A>=threshold] = 1
      A[A<threshold] = 0
      B[B>=threshold] = 1
      B[B<threshold] = 0
      result = (A + B)
      #binary_res = np.logical_and(result[:,:,0], result[:,:,1], result[:,:,2])
      binary_res = np.max(result, axis=2)


      True_mask = MyHoleMask[num].cpu().detach().numpy()
      res = (2 * True_mask-binary_res)
      maskTpreF = np.count_nonzero(res == 2)
      maskTpreT = np.count_nonzero(res == 1)
      maskFpreF = np.count_nonzero(res == 0)
      maskFpreT = np.count_nonzero(res == -1.0)

      Method1Acc.append(maskTpreT/(maskTpreT + maskTpreF))
      Method2Acc.append(maskTpreT/(maskTpreT + maskFpreT))
      Method3Acc.append((maskTpreT + maskFpreF)/(maskTpreT + maskFpreF + maskTpreF + maskFpreT))

    self.Method1AccHole.append(sum(Method1Acc)/len(Method1Acc))
    self.Method2AccHole.append(sum(Method2Acc)/len(Method2Acc))
    self.Method3AccHole.append(sum(Method3Acc)/len(Method3Acc))

  def cal_acc_print(self):
    Method1Acc = []
    Method2Acc = []
    Method3Acc = []
    for num in range(17):
      pred_img = self.Test_single(num, MyPrintDataSet)
      A = (pred_img - MyPrintDataSet[num]).view(128, 128, 3).cpu().detach().numpy()
      B = (MyPrintDataSet[num] - pred_img).view(128, 128, 3).cpu().detach().numpy()
      threshold = 0.1
      A[A>=threshold] = 1
      A[A<threshold] = 0
      B[B>=threshold] = 1
      B[B<threshold] = 0
      result = (A + B)
      #binary_res = np.logical_and(result[:,:,0], result[:,:,1], result[:,:,2])
      binary_res = np.max(result, axis=2)


      True_mask = MyPrintMask[num].cpu().detach().numpy()
      res = (2 * True_mask-binary_res)
      maskTpreF = np.count_nonzero(res == 2)
      maskTpreT = np.count_nonzero(res == 1)
      maskFpreF = np.count_nonzero(res == 0)
      maskFpreT = np.count_nonzero(res == -1.0)

      Method1Acc.append(maskTpreT/(maskTpreT + maskTpreF))
      Method2Acc.append(maskTpreT/(maskTpreT + maskFpreT))
      Method3Acc.append((maskTpreT + maskFpreF)/(maskTpreT + maskFpreF + maskTpreF + maskFpreT))

    self.Method1AccPrint.append(sum(Method1Acc)/len(Method1Acc))
    self.Method2AccPrint.append(sum(Method2Acc)/len(Method2Acc))
    self.Method3AccPrint.append(sum(Method3Acc)/len(Method3Acc))

In [ ]:
MyNetwork = CNN_Net()
MyNetwork.to(device)


CNN_Net(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(32, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(64, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(64, eps=1e-06, momentum=0.1, affine=True, track_run

In [ ]:
MyNetwork.Train_net(learning_rate=0.00002, epoch_number=800)

In [ ]:
MyNetwork.plot_res()

In [ ]:
name = ["crack", "cut", "hole", "print"]
num_img = [18, 17, 18, 17]

num = 1
pred_img = MyNetwork.Test_single_img(num, MyPrintDataSet)
A = (pred_img - MyPrintDataSet[num]).view(128, 128, 3).cpu().detach().numpy()
B = (MyPrintDataSet[num] - pred_img).view(128, 128, 3).cpu().detach().numpy()

threshold = 0.3
A[A>=threshold] = 1
A[A<threshold] = 0
B[B>=threshold] = 1
B[B<threshold] = 0

result = (A + B)
plt.imshow(result)
plt.show()

#binary_res = np.logical_and(result[:,:,0], result[:,:,1], result[:,:,2])
binary_res = np.max(result, axis=2)
plt.imshow(binary_res)
plt.show()

True_mask = MyPrintMask[num].cpu().detach().numpy()
plt.imshow(True_mask)
plt.show()

res = (2 * True_mask-binary_res)
maskTpreF = np.count_nonzero(res == 2)
maskTpreT = np.count_nonzero(res == 1)
maskFpreF = np.count_nonzero(res == 0)
maskFpreT = np.count_nonzero(res == -1.0)



Method1Acc = (maskTpreT/(maskTpreT + maskTpreF))
Method2Acc = (maskTpreT/(maskTpreT + maskFpreT))
Method3Acc = ((maskTpreT + maskFpreF)/(maskTpreT + maskFpreF + maskTpreF + maskFpreT))

print(Method1Acc)
print(Method2Acc)
print(Method3Acc)